In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist

# --- 1. DATA SPLIT ---
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

# Further split training into train + validation
x_train, x_val = x_train[:50000], x_train[50000:]
y_train, y_val = y_train[:50000], y_train[50000:]

# --- 2. DEFINE THREE DIFFERENT ARCHITECTURES ---
def build_model(arch):
    if arch == 1:  # simple
        model = models.Sequential([
            layers.Flatten(input_shape=(28,28)),
            layers.Dense(64, activation='relu'),
            layers.Dense(10, activation='softmax')
        ])
    elif arch == 2:  # deeper
        model = models.Sequential([
            layers.Flatten(input_shape=(28,28)),
            layers.Dense(128, activation='relu'),
            layers.Dense(64, activation='relu'),
            layers.Dense(10, activation='softmax')
        ])
    elif arch == 3:  # with dropout
        model = models.Sequential([
            layers.Flatten(input_shape=(28,28)),
            layers.Dense(128, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(10, activation='softmax')
        ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# --- 3. TRAIN AND EVALUATE ON VALIDATION ---
val_scores = []
for arch in [1,2,3]:
    print(f"\nTraining Architecture {arch}")
    model = build_model(arch)
    model.fit(x_train, y_train, epochs=5, batch_size=32, verbose=0)
    
    # Compare using VALIDATION SET
    val_loss, val_acc = model.evaluate(x_val, y_val, verbose=0)
    print(f"Architecture {arch} Validation Accuracy: {val_acc}")
    val_scores.append((arch, val_acc, model))

# --- 4. SELECT BEST MODEL ---
best_arch, best_val_acc, best_model = max(val_scores, key=lambda x: x[1])
print(f"\nBest architecture based on validation: {best_arch} ({best_val_acc:.4f})")

# --- 5. FINAL TEST ON UNSEEN TEST SET ---
test_loss, test_acc = best_model.evaluate(x_test, y_test, verbose=0)
print(f"Final Test Accuracy (unbiased): {test_acc:.4f}")



Training Architecture 1


C:\Users\Shantanu Khosla\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Architecture 1 Validation Accuracy: 0.9695000052452087

Training Architecture 2
Architecture 2 Validation Accuracy: 0.972100019454956

Training Architecture 3
Architecture 3 Validation Accuracy: 0.9724000096321106

Best architecture based on validation: 3 (0.9724)
Final Test Accuracy (unbiased): 0.9705
